# Generating CLEAN Results

## Load environment

In [1]:
%matplotlib inline
import sys

# Directories and paths
lib_path = '/gpfswork/rech/xdy/uze68md/GitHub/'
data_path = '/gpfswork/rech/xdy/uze68md/data/'
model_dir = '/gpfswork/rech/xdy/uze68md/trained_models/model_cfht/'

# Add library path to PYTHONPATH
path_alphatransform = lib_path+'alpha-transform'
path_score = lib_path+'score'
path_clean = '/gpfswork/rech/xdy/uze68md/GitHub/ShapeDeconv/data/T-RECS/clean/'
sys.path.insert(0, path_alphatransform)
sys.path.insert(0, path_score)
sys.path.insert(0, path_clean)

# Function

def sigma_mad(signal):
    """This function returns the estimate of the standard deviation of White
    Additive Gaussian Noise using the Mean Absolute Deviation method (MAD).
    INPUT: signal, Numpy Array
    OUTPUT: sigma, scalar"""
    sigma = 1.4826*np.median(np.abs(signal-np.median(signal)))
    return sigma

# Libraries
from CLEAN3 import doCLEAN,gauss2D
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import fft
import cadmos_lib as cl
import tensorflow as tf
import galsim
from galsim import Image
import galsim.hsm
import galflow as gf
from galaxy2galaxy import problems
import pickle
from cv2 import resize, INTER_AREA
from scipy.signal import convolve

## Load Data

In [2]:
# Load Data
f = open(data_path+"meerkat_batch.pkl", "rb")
batch = pickle.load(f)
f.close()

# Normalize PSF by the max
batch['psf'] = np.array([psf/np.max(psf) for psf in batch['psf']])

n_batch, Nx, Ny = batch['targets'].shape

## Apply CLEAN

In [3]:
restored,restorednores,residual,skymodelimg = np.zeros((4,n_batch,Nx,Ny))
skymodellist = []
sigma_flags = np.ones(n_batch,dtype=bool)
for i in range(n_batch):
    # running CLEAN
    sigma_im = sigma_mad(batch['inputs'][i])
    n=3
    while np.max(batch['inputs'][i])<n*sigma_im:
        n = n-1
        sigma_flags[i] = False
    if  i%100 == 0:
        print(i)
        print('n :',n)
    restored[i],restorednores[i],residual[i],skymodellist_temp,skymodelimg[i]=doCLEAN(batch['inputs'][i],batch['psf'][i],gain=0.1,niter=5000,fthresh=n*sigma_im)
    skymodellist += [skymodellist_temp]

0
n : 3
100
n : 3
200
n : 3
300
n : 3
400
n : 3
500
n : 3
600
n : 3
700
n : 3
800
n : 3
900
n : 3
1000
n : 3
1100
n : 3
1200
n : 3
1300
n : 3
1400
n : 3
1500
n : 3
1600
n : 3
1700
n : 3
1800
n : 3
1900
n : 3
2000
n : 3
2100
n : 2
2200
n : 3
2300
n : 3
2400
n : 3
2500
n : 3
2600
n : 3
2700
n : 3
2800
n : 3
2900
n : 3
3000
n : 3


## Reconstruct with a Gaussian PSF

In [4]:
# Generate isotropic Gaussian PSF
# preparing gridding
imsize = 128
x=np.arange(0,imsize)-imsize//2
y=np.arange(0,imsize)-imsize//2
xx,yy=np.meshgrid(x,y)
gauss_iso = gauss2D(xx,yy,1,0,0,2,0.,0.)  # gridx, gridy, amp, meanx, meany, std, e1, e2

# normalize Gaussian PSF with the flux of the PSF
gauss_psf = np.array([gauss_iso / gauss_iso.sum() * psf.sum() for psf in batch['psf']])

restored_iso = np.array([convolve(sm,psf,'same') for sm,psf in zip(skymodelimg,gauss_psf)])

## Build Result Dictionnary

In [5]:
keys = ['restored_residual', 'restored', 'restored_isotropic', 'residual', 'skymodel', 'skymodel_list', 'sigma_flags']
values = [restored,restorednores,restored_iso,residual,skymodelimg,skymodellist,sigma_flags]

results = {}

for key,value in zip(keys,values):
    results[key] = value

## Save Result Dictionnary

In [6]:
f = open(data_path+"clean_results.pkl","wb")
pickle.dump(results,f)
f.close()